# NLP Pipeline Prototyping

Load and preprocess FB & TW data.

In [1]:
import os, re

import numpy as np
import pandas as pd

import nltk
nltk.data.path.append("/home/datadonk23/WDir/NLP/nltk_data")
from nltk import word_tokenize

import matplotlib.pyplot as plt
%matplotlib inline

## Loading

Paths

In [2]:
data_path = "/mnt/DATA/NRW2019 Dropbox/data 4good/CSVData"

fb_posts_path = os.path.join(data_path, "FBPolTimeLines.csv")
fb_comments_path = os.path.join(data_path, "FBUserComments.csv")
twitter_posts_path = os.path.join(data_path, "TwPolTimeLines.csv")
twitter_comments_path = os.path.join(data_path, "TwUserComments.csv")

Load datasets

In [3]:
fb_posts = pd.read_csv(fb_posts_path)
fb_posts["media_source"] = "fb"
fb_comments = pd.read_csv(fb_comments_path)
fb_comments["media_source"] = "fb"

twitter_posts = pd.read_csv(twitter_posts_path)
twitter_posts["media_source"] = "tw"
twitter_comments = pd.read_csv(twitter_comments_path)
twitter_comments["media_source"] = "tw"

Merge datasets

In [4]:
raw_data = pd.concat([fb_posts, fb_comments, twitter_posts, twitter_comments], sort=False)
assert raw_data.shape[0] == (fb_posts.shape[0] + fb_comments.shape[0] +
                             twitter_posts.shape[0] + twitter_comments.shape[0]), "False samples count"

print("Shape:", raw_data.shape)

Shape: (172047, 41)


## Preprocessing

Drop rows w/o text

In [5]:
data = raw_data.dropna(subset=["text"], how="all")
print("Removed rows:", raw_data.shape[0] - data.shape[0])

Removed rows: 3533


Remove unnecessary columns

In [6]:
#data.columns

Preprocess text:
* Lowercase ?
* Numbers ?
* Mentions
* Hashtags

In [7]:
# data["cleaned_text"] = 

Tokenize text

In [8]:
# data["tokens"] = data.cleaned_text.apply(word_tokenize)

## Dump cleaned dataset

To use for Sentiment Analysis and Topic Modelling.

In [9]:
dump_path = os.path.join(data_path, "CleanedTextDF.csv")
#data.to_csv(dump_path, index=False)